# Scikit-learn hyperparameter search wrapper

Iaroslav Shcherbatyi, Tim Head and Gilles Louppe. June 2017.

## Introduction

This example assumes basic familiarity with [scikit-learn](http://scikit-learn.org/stable/index.html). 

Search for parameters of machine learning models that result in best cross-validation performance is necessary in almost all practical cases to get a model with best generalization estimate. A standard approach in scikit-learn is using `GridSearchCV` class, which takes a set of values for every parameter to try, and simply enumerates all combinations of parameter values. The complexity of such search grows exponentially with addition of new parameters. A more scalable approach is using `RandomizedSearchCV`, which however does not take advantage of the structure of a search space.

Scikit-optimize provides a drop in replacement for `GridSearchCV`, which utilizes Bayesian Optimization where a predictive model reffered to as "surrogate" is used to model the search space and utilized in order to arrive at good parameter values combination as soon as possible.


## Minimal example
 
A minimalistic example of optimizing hyperparameters of SVC (Support Vector machine Classifier) is given below.


In [5]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    SVC(),
    [{
        'C': Real(1e-6, 1e+6, prior='log-uniform'),  
        'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
        'degree': Integer(1,8),
        'kernel': Categorical(['linear', 'poly', 'rbf']),
    }],
    n_jobs=1, n_iter=32
)

opt.fit(X_train, y_train)
print(opt.score(X_test, y_test))

0.973684210526


## Advanced example 

In practice, one wants to enumerate over multiple predictive model classes, with different search spaces and number of evaluations per class. An example of such search over parameters of Linear SVM, Kernel SVM and decision trees is given below. 

In [4]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.datasets import load_iris
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

# used to try different model classes
pipe = Pipeline([
    ('model', SVC())
])

# single categorical value of 'model' parameter is used  to set the model class
lin_search = {
    'model': Categorical([LinearSVC()]),
    'model__C': Real(1e-6, 1e+6, prior='log-uniform'),
}

dtc_search = {
    'model': Categorical([DecisionTreeClassifier()]),
    'model__max_depth': Integer(1,32),
    'model__min_samples_split': Real(1e-3, 1.0, prior='log-uniform'),
}

svc_search = {
    'model': Categorical([SVC()]),
    'model__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'model__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'model__degree': Integer(1,8),
    'model__kernel': Categorical(['linear', 'poly', 'rbf']),
}

opt = BayesSearchCV(
    pipe,
    [(lin_search, 16), (dtc_search, 24), (svc_search, 32)], # (parameter space, # of evaluations)
)

opt.fit(X_train, y_train)
print(opt.score(X_test, y_test))

0.973684210526


## Iterative search utilizing `step` function

A single call to `step` function of the `BayesSearchCV` instance allows to perform partial search for at most `n_jobs` points in parallel. This allows to use custom stopping criterions and to pickle a parameter search class instance to the file for recovery in case of failures of any sort or in case computations need to be transported to a different machine. 

An example usage is shown below. 

In [5]:
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

search_space = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),
    'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'degree': Integer(1,8),
    'kernel': Categorical(['linear', 'poly', 'rbf']),
}

opt = BayesSearchCV(SVC())

# provide instance of search space 
opt.add_spaces(search_space, 'svc')

for i in range(16):
    opt.step(X_train, y_train, 'svc')
    # save the model or use custom stopping criterion here
    # model is updated after every step
    # ...
    score = opt.score(X_test, y_test)
    print(i, score)

0 0.236842105263
1 0.236842105263
2 0.236842105263
3 0.973684210526
4 0.973684210526
5 0.973684210526
6 0.973684210526
7 0.973684210526
8 0.973684210526
9 0.973684210526
10 0.973684210526
11 0.973684210526
12 0.973684210526
13 0.973684210526
14 0.973684210526
15 0.973684210526
